In [1]:
# !pip install python-dotenv
# !pip install openai

  Using cached openai-0.27.8-py3-none-any.whl (73 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached aiohttp-3.8.4-cp38-cp38-macosx_11_0_arm64.whl (337 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.0/123.0 kB 541.2 kB/s eta 0:00:00a 0:00:01
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached urllib3-2.0.3-py3-none-any.whl (123 kB)
  Using cached certifi-2023.5.7-py3-none-any.whl (156 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached multidict-6.0.4-cp38-cp38-macosx_11_0_arm64.whl (29 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached yarl-1.9.2-cp38-cp38-macosx_11_0_arm64.whl (62 kB)
  Using cached frozenlist-1.3.3-cp38-cp38-macosx_11_0_arm64.whl (35 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)


In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [6]:
# !pip install --upgrade langchain

In [7]:
from langchain.chat_models import ChatOpenAI

# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0)

In [53]:
template_string = """
    {final_text}

    1. What are the major diagnoses in the above admission. 

    {format_instructions}
    """

In [54]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [55]:
print(prompt_template.messages[0].prompt)
print(prompt_template.messages[0].prompt.input_variables)

input_variables=['final_text', 'format_instructions'] output_parser=None partial_variables={} template='\n    {final_text}\n\n    1. What are the major diagnoses in the above admission. \n\n    {format_instructions}\n    ' template_format='f-string' validate_template=True
['final_text', 'format_instructions']


In [77]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

diagnosis = ResponseSchema(name="Diagnosis",
                             description="Diagnosis extracted from the patient note")
icd10 = ResponseSchema(name="ICD10",
                                      description="ICD10 code of the diagnosis")
primary_diagnosis = ResponseSchema(name="Primary Diagnosis",
                                    description="Whether the given diagnosis is the primary diagnosis")

response_schemas = [diagnosis, 
                    icd10,
                    primary_diagnosis]

In [85]:
STRUCTURED_FORMAT_INSTRUCTIONS = """The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
[{{
{format}
}}]
```"""

line_template = '\t"{name}": {type}  // {description}'

def _get_sub_string(schema: ResponseSchema) -> str:
    return line_template.format(
        name=schema.name, description=schema.description, type=schema.type
    )

class CustomOutputParser(StructuredOutputParser):

    def get_format_instructions(self) -> str:
        schema_str = "\n".join(
            [_get_sub_string(schema) for schema in self.response_schemas]
        )
        return STRUCTURED_FORMAT_INSTRUCTIONS.format(format=schema_str)

In [87]:
output_parser = CustomOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [90]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
[{
	"Diagnosis": string  // Diagnosis extracted from the patient note
	"ICD10": string  // ICD10 code of the diagnosis
	"Primary Diagnosis": string  // Whether the given diagnosis is the primary diagnosis
}]
```


In [91]:
prompt = ChatPromptTemplate.from_template(template=template_string)
messages = prompt.format_messages(final_text='Patient admitted with CHF and UTI', 
                                format_instructions=format_instructions)

In [30]:
import pandas as pd
dx_json = [{
    'Diagnosis':'CHF',
    'ICD10':'I50.9',
    'Primary Diagnosis':True
}]
pd.DataFrame(dx_json)

,Diagnosis,ICD10,Primary Diagnosis
0,CHF,I50.9,True


In [92]:
response = chat(messages)

In [93]:
output_dict = output_parser.parse(response.content)

OutputParserException: Got invalid return object. Expected key `Diagnosis` to be present, but got [{'Diagnosis': 'CHF', 'ICD10': 'I50.9', 'Primary Diagnosis': 'Yes'}, {'Diagnosis': 'UTI', 'ICD10': 'N39.0', 'Primary Diagnosis': 'No'}]

In [72]:
print(response.content)

```json
{
	"Diagnosis": "CHF",
	"ICD10": "I50.9",
	"Primary Diagnosis": "Yes"
},
{
	"Diagnosis": "UTI",
	"ICD10": "N39.0",
	"Primary Diagnosis": "No"
}
```


In [74]:
print(messages[0].content)


    Patient admitted with CHF and UTI

    1. What are the major diagnoses in the above admission. 

    The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Diagnosis": string  // Diagnosis extracted from the patient note
	"ICD10": string  // ICD10 code of the diagnosis
	"Primary Diagnosis": string  // Whether the given diagnosis is the primary diagnosis
}
```
    
